In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv



## Importing the required libraries


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

## Reading the datasets

The datasets has over 5 million rows, working with all the rows at once, might becomne a bit problematic and slow down our device, for this initial prediction, i will be working with 1% of the total training data which should be 500,000 plus rows

In [3]:
#Loading the train_df
#The training data path
train_path = '/kaggle/input/new-york-city-taxi-fare-prediction/train.csv'

#Defining constants
sample_fraction = 0.01
random.seed(92)

#Defining the function to randomly sample 1 percent of the data
def skip_row(idx):
    if idx == 0:
        return False
    return random.random() > sample_fraction

#Loading the df
df = pd.read_csv(train_path, parse_dates = ['pickup_datetime'], skiprows = skip_row)

In [4]:
#Viewing the first 5 rows of the data frame
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3


In [5]:
#Loading the test_df
#The test data path
test_path = '/kaggle/input/new-york-city-taxi-fare-prediction/test.csv'

#Loading the df
test_df = pd.read_csv(test_path, parse_dates = ['pickup_datetime'],)

## Exploring the datasets

In [6]:
#Viewing info about the training data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553345 entries, 0 to 553344
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   key                553345 non-null  object             
 1   fare_amount        553345 non-null  float64            
 2   pickup_datetime    553345 non-null  datetime64[ns, UTC]
 3   pickup_longitude   553345 non-null  float64            
 4   pickup_latitude    553345 non-null  float64            
 5   dropoff_longitude  553344 non-null  float64            
 6   dropoff_latitude   553344 non-null  float64            
 7   passenger_count    553345 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 33.8+ MB


In [7]:
#Statististical summary of the training datset
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,553345.000000,553345.000000,553345.000000,553344.000000,553344.000000,553345.000000
mean,11.358222,-72.509886,39.906591,-72.472268,39.913447,1.684567
std,9.833439,13.800688,10.425359,13.500015,11.529377,1.308479
min,-52.000000,-3313.387292,-3457.346617,-928.225348,-2909.234763,0.000000
25%,6.000000,-73.992037,40.734968,-73.991401,40.734028,1.000000
50%,8.500000,-73.981755,40.752672,-73.980145,40.753132,1.000000
75%,12.500000,-73.966973,40.767177,-73.963604,40.768109,2.000000
max,411.110000,2551.566137,2973.980450,3211.579750,3333.304575,9.000000


In [8]:
#Checking the min and highest date in the training df
df['pickup_datetime'].min(), df['pickup_datetime'].max()

(Timestamp('2009-01-01 00:15:42+0000', tz='UTC'),
 Timestamp('2015-06-30 23:56:38+0000', tz='UTC'))

In [9]:
#info on the test set
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                9914 non-null   object             
 1   pickup_datetime    9914 non-null   datetime64[ns, UTC]
 2   pickup_longitude   9914 non-null   float64            
 3   pickup_latitude    9914 non-null   float64            
 4   dropoff_longitude  9914 non-null   float64            
 5   dropoff_latitude   9914 non-null   float64            
 6   passenger_count    9914 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 542.3+ KB


In [10]:
#Statistical summary of the test data
test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [11]:
#Checking the min and highest date in the test df
test_df['pickup_datetime'].min(), test_df['pickup_datetime'].max()

(Timestamp('2009-01-01 11:04:24+0000', tz='UTC'),
 Timestamp('2015-06-30 20:03:50+0000', tz='UTC'))

#### Observations
* 550k plus rows
* No missing data in the df
* Key column is not neeeded
* There are negative values in the longitude and latitude columns which is unsual
* The passener count column has some unusally large numbers
* Data takes 30+ mb in storage
* Data from both the train and test starts from 2009 and ends in 2015
#### We will need to deal with outliers before training the model

## Preparing the data for Training


In [12]:
#Viewing the colums in the df
df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [25]:
#Dropping missng Values
df = df.dropna()

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 553344 entries, 0 to 553344
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   key                553344 non-null  object             
 1   fare_amount        553344 non-null  float64            
 2   pickup_datetime    553344 non-null  datetime64[ns, UTC]
 3   pickup_longitude   553344 non-null  float64            
 4   pickup_latitude    553344 non-null  float64            
 5   dropoff_longitude  553344 non-null  float64            
 6   dropoff_latitude   553344 non-null  float64            
 7   passenger_count    553344 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 38.0+ MB


In [27]:
#Splitting the training data into features and labels
X = df.drop(['key', 'fare_amount','pickup_datetime'], axis = 1)
y = df['fare_amount']

In [14]:
#Subsetting the test data
test = test_df.drop(['key','pickup_datetime'], axis = 1)

In [29]:
#Splitting the data into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state = 92)

In [30]:
#Validating the split
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(442675, 5) (110669, 5)
(442675,) (110669,)


## Training a Baseline Model

In [31]:
#Mean Regressor 
class MeanRegressor:
    def fit(self, inputs, targets):
        self.mean = targets.mean()
        
    def predict(self, inputs):
        return np.full(inputs.shape[0], self.mean)

In [48]:
#Instantiating the model
mean_model = MeanRegressor()

#Fitting the model to the input data
mean_model.fit(X_train, y_train)

#Predicting on the train data
train_pred = mean_model.predict(X_train)

#Predicting on the val data
val_pred = mean_model.predict(X_val)


## Evaluating the Baseline Model

In [33]:
#Defining the evaluation function

def rmse(targets, pred):
    return mean_squared_error(targets, pred, squared = False)


In [49]:
#Train rmse
train_rmse = rmse(y_train, train_pred)


#Validation rmse
val_rmse = rmse(y_val, val_pred)

print(f"The Root mean squared error of the baseline model on the train data is: {train_rmse}")
print(f"The Root mean squared erorr of the baseline model on the val data is: {val_rmse}")

The Root mean squared error of the baseline model on the train data is: 9.826516523872696
The Root mean squared erorr of the baseline model on the val data is: 9.861092257690506


### The Hard coded model is off by ~9.8, this is not a good result

## Training a Linear Regression Model

In [35]:
#Instantiating the model
l_reg = LinearRegression()

In [36]:
#Fitting the model to the train data
l_reg.fit(X_train,y_train)

#Making predictions on the train data
train_pred = l_reg.predict(X_train)

#Making predictions on the val data
val_pred = l_reg.predict(X_val)

## Evaluating the Linear Regression Model

In [37]:
#RMSE on the train prediction
train_rmse = rmse(y_train, train_pred)

#RMSE on the val prediction
val_rmse = rmse(y_val, val_pred)

print(f"The RMSE of the model on the train data is: {train_rmse}")
print(f"The RMSE of the model on the val data is: {val_rmse}")

The RMSE of the model on the train data is: 9.824843578269924
The RMSE of the model on the val data is: 9.858982370635724


### The RMSE of the Linear Regression model is not better than what we achieved with our average model

## Predictions for Initial submission to Kaggle

In [38]:
#Making predictions on the test data
test_pred = l_reg.predict(test)

In [39]:
#Preparing the submission file
sub_dir = '/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv'
submission = pd.read_csv(sub_dir)

#Viewing the file
submission

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.35
1,2015-01-27 13:08:24.0000003,11.35
2,2011-10-08 11:53:44.0000002,11.35
3,2012-12-01 21:12:12.0000002,11.35
4,2012-12-01 21:12:12.0000003,11.35
...,...,...
9909,2015-05-10 12:37:51.0000002,11.35
9910,2015-01-12 17:05:51.0000001,11.35
9911,2015-04-19 20:44:15.0000001,11.35
9912,2015-01-31 01:05:19.0000005,11.35


In [40]:
#Subtituting the values
submission['fare_amount'] = test_pred

In [41]:
#Saving the file to csv
submission.to_csv('1st_submission.csv', index = None)

In [44]:
#Defining a function for predicting and saving the submission file
def make_predictions_and_submit(model, test_input, file_name):
    test_pred = model.predict(test_input)
    sub_df = pd.read_csv(sub_dir)
    sub_df['fare_amount'] = test_pred
    sub_df.to_csv(file_name, index = None)
    return sub_df

## Feature Engineering

In [51]:
#Parsing the date column
def parse_date(df, col):
    df[col + '_year'] = df[col].dt.year
    df[col + '_month'] = df[col].dt.month
    df[col + '_day'] = df[col].dt.day
    df[col + '_weekday'] = df[col].dt.weekday
    df[col + '_hour'] = df[col].dt.hour

In [53]:
#Parsing the date column for the training data
parse_date(df, 'pickup_datetime')

In [55]:
#Parsing the date column for the test dat
parse_date(test_df, 'pickup_datetime')

In [58]:
train_df = df.copy()

In [59]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1,2009,3,2,0,20
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6,2015,4,12,6,10
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1,2012,11,6,1,11
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5,2012,10,6,5,15
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3,2015,5,31,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
553340,2010-08-14 21:00:17.0000002,10.9,2010-08-14 21:00:17+00:00,-73.966080,40.805193,-73.954060,40.781668,1,2010,8,14,5,21
553341,2012-04-20 09:31:00.00000039,5.7,2012-04-20 09:31:00+00:00,-73.977957,40.748792,-73.989132,40.742062,1,2012,4,20,4,9
553342,2013-01-21 14:17:17.0000002,7.5,2013-01-21 14:17:17+00:00,-73.999356,40.740943,-73.984007,40.746730,1,2013,1,21,0,14
553343,2013-02-20 08:25:20.0000001,11.5,2013-02-20 08:25:20+00:00,-73.983433,40.738376,-73.975043,40.760101,1,2013,2,20,2,8


In [57]:
#Creating a fucntion for calculating the distance between the pickup and drop off location
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [63]:
#Creating a fucntion for calculating the distance between the pickup and drop off location
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [60]:
train_df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'pickup_datetime_year', 'pickup_datetime_month',
       'pickup_datetime_day', 'pickup_datetime_weekday',
       'pickup_datetime_hour'],
      dtype='object')

In [64]:
#Creating a function to add the distance to the dataframe
def add_distance(df):
    df['trip_distance'] = haversine_np(df['pickup_longitude'],
                                    df['pickup_latitude'],
                                    df['dropoff_longitude'],
                                    df['dropoff_latitude']
                                   )

In [65]:
#Adding trip distance to the train data
add_distance(train_df)
#Adding trip distance to test data
add_distance(test_df)

In [66]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1,2009,3,2,0,20,3.583739
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6,2015,4,12,6,10,3.372594
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1,2012,11,6,1,11,0.564851
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5,2012,10,6,5,15,1.798889
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3,2015,5,31,6,7,6.362372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553340,2010-08-14 21:00:17.0000002,10.9,2010-08-14 21:00:17+00:00,-73.966080,40.805193,-73.954060,40.781668,1,2010,8,14,5,21,2.802987
553341,2012-04-20 09:31:00.00000039,5.7,2012-04-20 09:31:00+00:00,-73.977957,40.748792,-73.989132,40.742062,1,2012,4,20,4,9,1.201860
553342,2013-01-21 14:17:17.0000002,7.5,2013-01-21 14:17:17+00:00,-73.999356,40.740943,-73.984007,40.746730,1,2013,1,21,0,14,1.443436
553343,2013-02-20 08:25:20.0000001,11.5,2013-02-20 08:25:20+00:00,-73.983433,40.738376,-73.975043,40.760101,1,2013,2,20,2,8,2.515395


In [ ]:
#Adding distance from popular landmark
#JFK Airport
jfk = -73.7781, 40.6413

#LGA Airport
lga = -73.8740, 40.7769

#EWR Ariport
ewr = -74.175, 40.6895

#MET Museum
met = -74.9632, 40.7794

#World Trade Centre
wtc = -74.0099, 40.7126


In [ ]:
#Creating a Function to add the distance from a landmark to the pickup location
def add_landmark_pickup_distance(df, landmark, lonlat):
    lon, lat = lonlat
    df[landmark + 'pickup_distance'] = haversine_np(lon, lat, 
                                                    df['pickup_longitude'], 
                                                    df['pickup_latitude'])